In [7]:
def min_edit_script(source, target, allow_copy=False):
    """
    Finds the minimum edit script to transform the source to the target
    """
    a = [[(len(source) + len(target) + 1, None)] * (len(target) + 1) for _ in range(len(source) + 1)]
    for i in range(0, len(source) + 1):
        for j in range(0, len(target) + 1):
            if i == 0 and j == 0:
                a[i][j] = (0, "")
            else:
                if allow_copy and i and j and source[i - 1] == target[j - 1] and a[i-1][j-1][0] < a[i][j][0]:
                    a[i][j] = (a[i-1][j-1][0], a[i-1][j-1][1] + "→")
                if i and a[i-1][j][0] < a[i][j][0]:
                    a[i][j] = (a[i-1][j][0] + 1, a[i-1][j][1] + "-")
                if j and a[i][j-1][0] < a[i][j][0]:
                    a[i][j] = (a[i][j-1][0] + 1, a[i][j-1][1] + "+" + target[j - 1])
    return a[-1][-1][1]

In [8]:
min_edit_script("Mines","miabnesed",allow_copy=True)

'-+m→+a+b→→→+e+d'

In [1]:
def short_edit_script(source, target, allow_copy=False):
    """
    Finds the minimum edit script to transform the source to the target
    source->form, target->lemma
    """
    a = [[(len(source) + len(target) + 1, None)] * (len(target) + 1) for _ in range(len(source) + 1)]
    for i in range(0, len(source) + 1):
        for j in range(0, len(target) + 1):
            if i == 0 and j == 0:
                a[i][j] = (0, "")
            else:
                if allow_copy and i and j and source[i - 1].lower() == target[j - 1] and a[i-1][j-1][0] < a[i][j][0]:
                    if source[i - 1] == target[j - 1]:
                        a[i][j] = (a[i-1][j-1][0], a[i-1][j-1][1] + "→")
                    else:
                        a[i][j] = (a[i-1][j-1][0], a[i-1][j-1][1] + "↓")
                if i and a[i-1][j][0] < a[i][j][0]:
                    a[i][j] = (a[i-1][j][0] + 1, a[i-1][j][1] + "-")
                if j and a[i][j-1][0] < a[i][j][0]:
                    a[i][j] = (a[i][j-1][0] + 1, a[i][j-1][1] + "+" + target[j - 1])
    return a[-1][-1][1]

In [2]:
short_edit_script("Mines","miabnesed",allow_copy=False)

'-----+m+i+a+b+n+e+s+e+d'

In [3]:
short_edit_script("Mines","miabnesed",allow_copy=True)

'↓→+a+b→→→+e+d'

四个‘-’表示去掉w,e,n,然后加上g,o</br>
‘↓’表示把大写变小写，这里假设在通常情况下，form中的为小写的字母，在lemma中一定是小写，不可能是大写。

In [30]:
def gen_lemma_rule(form, lemma, allow_copy=False):
    """
    Generates a lemma rule to transform the form to the lemma
    """
    form = form.lower()

    previous_case = -1
    # lemma_casing is to divide lemma into upper case substring and lower case substring
    lemma_casing = ""
    for i, c in enumerate(lemma):
        case = "↑" if c.lower() != c else "↓"   # 判断lemma中的每个字母是大写还是小写
        if case != previous_case:
            lemma_casing += "{}{}{}".format("¦" if lemma_casing else "", case, i if i <= len(lemma) // 2 else i - len(lemma))   # ‘//’向下取整商，
        previous_case = case
    lemma = lemma.lower()

    best, best_form, best_lemma = 0, 0, 0
    # Find the same substring between form and lemma
    for l in range(len(lemma)):
        for f in range(len(form)):
            cpl = 0
            while f + cpl < len(form) and l + cpl < len(lemma) and form[f + cpl] == lemma[l + cpl]: cpl += 1
            if cpl > best:
                best = cpl
                best_form = f
                best_lemma = l

    rule = lemma_casing + ";"
    if not best:
        rule += "a" + lemma
    else:
        rule += "start:{}¦{}".format(
            form[:best],
            short_edit_script(form[best_form + best:], lemma[best_lemma + best:], allow_copy),
        )
    return rule

In [32]:
gen_lemma_rule('Students', 'Studeent', allow_copy=True)

'↑0¦↓1;start:stude¦+e→→-'

In [14]:
type([gen_lemma_rule('students', 'studant', allow_copy=False)])

list